In [1]:
import bolift
import numpy as np
import json
import pandas as pd
from langchain.prompts.prompt import PromptTemplate
import itertools
import os
import openai

np.random.seed(0)

# Ablation experiments

In [2]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def run_ablation_experiment(asktell, train_data, test_data):
    for i in range(len(train_data)):
        asktell.tell(train_data.iloc[i, 0], float(train_data.iloc[i, 1]))
    x    = []
    y    = []
    yhat = []
    for j in range(len(test_data)):
        x.append(test_data.iloc[j, 0])
        y.append(float(test_data.iloc[j, 1]))
        yhat.append(asktell.predict(test_data.iloc[j, 0]))

    # We should count how many x's have no yhat
    x_filter = [xi for xi, yhi in zip(x, yhat) if len(yhi.values) > 0]
    y_filter = [yi for yi, yhi in zip(y, yhat) if len(yhi.values) > 0]
    yhat_filter = [yhi for yi, yhi in zip(y, yhat) if len(yhi.values) > 0]
    return x_filter, y_filter, yhat_filter
    
def save_csv(filename, x, y, yhat, data, model, T, k, N, model_class, tokens):
    if not os.path.exists(filename):
        f = open(filename, "w")
        f.write("y;yhat;yprobs;data;model;Temperature;k_selected;N_train;model_class;n_tokens;x\n")
    else:
        f = open(filename, "a")
    for xi, yi, yhi in zip(x, y, yhat):
        for v,p in zip(yhi.values, yhi.probs):
            f.write(f"{yi};{v};{p:.4f};{data};{model};{T};{k};{N};{model_class};{tokens};{xi}\n")
    f.close()

T_list = [0.05, 0.5, 1.0, 1.5]
k_list = [0, 5, 10]
N_list = [1,2,3,4,5,10,25,50,100,200] # depends on each dataset
models_list = ["text-curie-001"]
out_csv_file = "ablation_results.csv"

In [3]:
# def parity_plot(y, yhat):
#     modes = [yhat[i].mode() for i in range(len(yhat))]
#     std = [yhat[i].std() for i in range(len(yhat))]
#     plt.errorbar(y, modes, yerr=std, fmt="o")
#     plt.plot(y, y)
#     plt.xlabel("Actual Yield")
#     plt.ylabel("Predicted Yield")
#     plt.show()

## C2 yield

In [18]:
data_path = "paper/data/C2_yield_meth_oxy_short.csv"
raw_data = pd.read_csv(data_path)

np.random.seed(0)

N = raw_data.shape[0]
train = np.random.choice(raw_data.shape[0], int(N * 0.8), replace=False)
test = np.setdiff1d(np.arange(raw_data.shape[0]), train)
np.random.shuffle(test)

train_data = raw_data.iloc[train, :].reset_index(drop=True)[:int(0.1*len(train))]
# train_data["prompt"] = train_data["prompt"].map(lambda x: x.replace(",", ";"))
test_data = raw_data.iloc[test, :].reset_index(drop=True)[:int(0.1*len(test))]
# test_data["prompt"] = test_data["prompt"].map(lambda x: x.replace(",", ";"))
print(N, len(train_data), len(test_data))

12708 1016 254


In [5]:
asktell = bolift.AskTellFewShotTopk(
    x_formatter=lambda x: f"experimental procedure: {x}",
    y_name="C2 yield",
    y_formatter=lambda y: f"{y:.2f}",
    model='text-curie-001',
    selector_k=1,
    temperature=0.05,
    verbose=False,
)

for i in range(25):
    asktell.tell(train_data.iloc[i, 0], float(train_data.iloc[i, 1]))
# test_data.iloc[j, 0]
# float(test_data.iloc[j, 1])
p = " To synthesize Mn-Na2WO4/Al2O3 , Al2O3 (1.0 g) was impregnated with 4.5 mL of an aqueous solution consiting of Mn ( 0.37 mol) , Na ( 0.37 mol) , W ( 0.185 mol) , at 50 �C for 6 h. Once activated the reaction is ran at 750 �C. The total flow rate was 15 mL/min (Ar: 6.0 mL/min, CH4: 6.8 mL/min, O2: 2.3 mL/min), leading to a contact time of 0.5 s."
asktell.predict(p)

 5.92
 5.92
 5.92
 5.92
 5.92


DiscreteDist([5.92 5.92 5.92 5.92 5.92], [0.2 0.2 0.2 0.2 0.2])

### multi

In [19]:
def run_C2_multi_ablation(train_data, test_data, model="text-curie-001", T=0.05, N=50, k=10):
    asktell = bolift.AskTellFewShotMulti(
        x_formatter=lambda x: f"experimental procedure: {x}",
        y_name="C2 yield",
        y_formatter=lambda y: f"{y:.2f}",
        model=model,
        selector_k=k,
        temperature=T
    )
    exp_train_data = train_data[:N]
    x, y, yhat =  run_ablation_experiment(asktell, exp_train_data, test_data)

    data="C2"
    model_class="multi"
    save_csv(out_csv_file, x, y, yhat, data, model, T, k, N, model_class, asktell.tokens_used)

    return y, yhat

In [20]:

T_list = [0.05]
k_list = [5]
N_list = [1000]
models_list = ["text-davinci-001"]
for T, k, N, model in itertools.product(T_list, k_list, N_list, models_list):
  print(f"Running C2 multi ablation with T={T}, k={k}, N={N}, model={model}", end=" ")
  y, yhat = run_C2_multi_ablation(train_data, test_data, model=model, T=T, N=N, k=k)
  print(" --> done")


Running C2 multi ablation with T=0.05, k=5, N=1000, model=text-davinci-001  --> done


### topk

In [21]:
def run_C2_topk_ablation(train_data, test_data, model="text-curie-001", T=0.05, N=50, k=10):
    asktell = bolift.AskTellFewShotTopk(
        x_formatter=lambda x: f"experimental procedure: {x}",
        y_name="C2 yield",
        y_formatter=lambda y: f"{y:.2f}",
        model=model,
        selector_k=k,
        temperature=T
    )
    exp_train_data = train_data[:N]
    x, y, yhat =  run_ablation_experiment(asktell, exp_train_data, test_data)

    data="C2"
    model_class="topk"
    save_csv(out_csv_file, x, y, yhat, data, model, T, k, N, model_class, asktell.tokens_used)

    return y, yhat

In [22]:

T_list = [0.05]
k_list = [5]
N_list = [1000]
models_list = ["text-davinci-001"]
for T, k, N, model in itertools.product(T_list, k_list, N_list, models_list):
  print(f"Running C2 topk ablation with T={T}, k={k}, N={N}, model={model}", end=" ")
  y, yhat = run_C2_topk_ablation(train_data, test_data, model=model, T=T, N=N, k=k)
  print(" --> done")


Running C2 topk ablation with T=0.05, k=5, N=1000, model=text-davinci-001  --> done


### Finetune

In [108]:
def run_C2_finetune(train_data, model="text-ada-001", N=50):
    asktell = bolift.AskTellFinetuning(
        prefix="",
        prompt_template=PromptTemplate(
            input_variables=["x", "y", "y_name"],
            template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
        ),
        suffix="What is the {y_name} of {x}?@@@\nA:",
        x_formatter=lambda x: f"experimental procedure: {x}",
        y_name="C2 yield",
        y_formatter=lambda y: f"{y:.2f}",
        model=model,
        n_epochs=8,
        learning_rate_multiplier=0.05,
    )
    # Tell one example so the moduel build the prompt
    asktell.tell(train_data.iloc[0, 0], train_data.iloc[0, 1])
    exp_train_data = train_data.iloc[:N]

    prompts=[]
    completions=[]
    for i in range(len(exp_train_data)):
        prompts.append(f"What is the yield strength of {exp_train_data.iloc[i, 0]}?@@@\\nA: ")
        completions.append(f"{float(exp_train_data.iloc[i, 1])}###")
    asktell.prepare_data(prompts, completions, f'./paper/out/data_C2_{N}.dat')
    asktell.fine_tune(prompts, completions, out_path='./paper/out', out_file=f'FT_C2_{N}')
    print(asktell.get_model_name())

def run_C2_FT_ablation(train_data, test_data, model="text-ada-001", T=0.05, N=10, k=0):
    with open(f'./paper/out/FT_C2_{N}.dat', 'r') as f:
        response = json.load(f)
    
    asktell = bolift.AskTellFinetuning(
        prefix="",
        prompt_template=PromptTemplate(
            input_variables=["x", "y", "y_name"],
            template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
        ),
        suffix="What is the {y_name} of {x}?@@@\nA:",
        # x_formatter=lambda x: f"alloy composition: {x}",
        y_name="C2 yield",
        y_formatter=lambda y: f"{y:.2f}",
        model=model,
        id=response['id'],
        selector_k=0,
        n_epochs=8,
        learning_rate_multiplier=0.05,
    )
    asktell.tell(train_data.iloc[0, 0], train_data.iloc[0, 1])
    exp_train_data = train_data.iloc[:1]
    x, y, yhat =  run_ablation_experiment(asktell, exp_train_data, test_data)

    data="C2"
    model_class="finetune"
    save_csv(out_csv_file, x, y, yhat, data, asktell.get_model_name(), T, k, N, model_class, asktell.tokens_used)

    return y, yhat

In [109]:
N_list=[500]
for N in N_list: 
  print(f"Running C2 FT with N={N}")
  run_C2_finetune(train_data, model="text-ada-001", N=N)
  print(" --> done")

Running C2 FT with N=500
text-ada-001
running.   Fine-tune succeeded                                                                

ada:ft-white-research-laboratory-2023-02-28-03-16-06
 --> done


In [110]:
T_list = [0.05]
k_list = [0]
N_list = [500]
models_list = ["text-ada-001"]
for T, k, N, model in itertools.product(T_list, k_list, N_list, models_list):
  print(f"Running C2 finetune ablation with T={T}, k={k}, N={N}, model={model}", end=" ")
  y, yhat = run_C2_FT_ablation(train_data, test_data, model="text-ada-001", T=T, N=N, k=k)
  print(" --> done")


Running C2 finetune ablation with T=0.05, k=0, N=500, model=text-ada-001 text-ada-001
ada:ft-white-research-laboratory-2023-02-28-03-16-06
 --> done


## Iupac-solubility

In [9]:
import requests
data_path = "paper/data/esol_iupac.csv"
raw_data = pd.read_csv(data_path)

def query2IUPAC(text):
  try:
    '''This function queries the one given molecule name and returns a SMILES string from the record'''
    #query the PubChem database
    r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/' + text + '/property/IUPACName/JSON')
    data = r.json()
    smi = data["PropertyTable"]["Properties"][0]["IUPACName"]
    return smi
  except:
    return None

# raw_data["IUPAC"] = raw_data["SMILES"].map(lambda sml: query2IUPAC(sml))
raw_data = raw_data[["IUPAC", "measured log(solubility:mol/L)"]]
raw_data = raw_data.dropna()

In [10]:
np.random.seed(0)

N = raw_data.shape[0]
train = np.random.choice(raw_data.shape[0], int(N * 0.8), replace=False)
test = np.setdiff1d(np.arange(raw_data.shape[0]), train)
np.random.shuffle(test)

train_data = raw_data.iloc[train, :].reset_index(drop=True)
test_data = raw_data.iloc[test, :].reset_index(drop=True)
print(N, len(train_data), len(test_data))

882 705 177


### multi

In [99]:
def run_iupac_sol_multi_ablation(train_data, test_data, model="text-curie-001", T=0.05, N=50, k=10):
    asktell = bolift.AskTellFewShotMulti(
        x_formatter=lambda x: f"iupac name {x}",
        y_name="measured log solubility in mols per litre",
        y_formatter=lambda y: f"{y:.2f}",
        model=model,
        selector_k=k,
        temperature=T
)
    exp_train_data = train_data[:N]
    x, y, yhat =  run_ablation_experiment(asktell, exp_train_data, test_data)

    data="iupac-sol"
    model_class="multi"
    save_csv(out_csv_file, x, y, yhat, data, model, T, k, N, model_class, asktell.tokens_used)

    return y, yhat

In [101]:

T_list = [0.05]
k_list = [5]
N_list = [250]
for T, k, N, model in itertools.product(T_list, k_list, N_list, models_list):
  print(f"Running iupac-solv multi ablation with T={T}, k={k}, N={N}, model={model}", end=" ")
  y, yhat = run_iupac_sol_multi_ablation(train_data, test_data, model="text-curie-001", T=T, N=N, k=k)
  print(" --> done")

Running iupac-solv multi ablation with T=0.05, k=5, N=250, model=text-curie-001  --> done


### topk

In [9]:
def run_iupac_sol_topk_ablation(train_data, test_data, model="text-curie-001", T=0.05, N=50, k=10):
    asktell = bolift.AskTellFewShotTopk(
        x_formatter=lambda x: f"iupac name {x}",
        y_name="measured log solubility in mols per litre",
        y_formatter=lambda y: f"{y:.2f}",
        model=model,
        selector_k=k,
        temperature=T
    )
    x, y, yhat =  run_ablation_experiment(asktell, train_data, test_data)

    N=len(train_data)
    data="iupac-sol"
    model_class="topk"
    save_csv(out_csv_file, x, y, yhat, data, model, T, k, N, model_class)

    return y, yhat

In [10]:
T_list = [0.05]
k_list = [5]
N_list = [5]
for T, k, N, model in itertools.product(T_list, k_list, N_list, models_list):
  print(f"Running iupac-solv multi ablation with T={T}, k={k}, N={N}, model={model}", end=" ")
  y, yhat = run_iupac_sol_topk_ablation(train_data, test_data, model="text-curie-001", T=T, N=N, k=k)
  print(" --> done")

Running iupac-solv multi ablation with T=0.05, k=5, N=5, model=text-curie-001 

RetryError: RetryError[<Future at 0x16d1db2b0 state=finished raised ValidationError>]

### Finetune

In [10]:
def run_iupac_sol_finetune(train_data, model="text-ada-001", N=50):
    asktell = bolift.AskTellFinetuning(
        prefix="",
        prompt_template=PromptTemplate(
            input_variables=["x", "y", "y_name"],
            template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
        ),
        suffix="What is the {y_name} of {x}?@@@\nA:",
        # x_formatter=lambda x: f"iupac name {x}",
        y_name="measured log solubility in mols per litre",
        y_formatter=lambda y: f"{y:.2f}",
        model=model,
        n_epochs=8,
        learning_rate_multiplier=0.02,
    )
    asktell.tell(train_data.iloc[0, 0], train_data.iloc[0, 1])
    exp_train_data = train_data.iloc[:N]

    prompts=[]
    completions=[]
    for i in range(len(train_data[:N])):
        prompts.append(f"What is the measured log solubility in mols per litre of {exp_train_data.iloc[i, 0]}?@@@\\nA: ")
        completions.append(f"{float(exp_train_data.iloc[i, 1])}###")
        asktell.prepare_data(prompts, completions, f'./paper/out/data_solv_{N}.dat')
    asktell.fine_tune(prompts, completions, out_path='./paper/out', out_file=f'FT_solv_{N}')
    print(asktell.get_model_name())

def run_iupac_sol_FT_ablation(train_data, test_data, model="text-ada-001", T=0.05, N=50, k=0):
    with open(f'./paper/out/FT_solv_{N}.dat', 'r') as f:
        response = json.load(f)
    
    asktell = bolift.AskTellFinetuning(
        prefix="",
        prompt_template=PromptTemplate(
            input_variables=["x", "y", "y_name"],
            template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
        ),
        suffix="What is the {y_name} of {x}?@@@\nA:",
        # x_formatter=lambda x: f"iupac name {x}",
        y_name="measured log solubility in mols per litre",
        y_formatter=lambda y: f"{y:.2f}",
        model=model,
        id=response['id'],
        selector_k=0,
        n_epochs=8,
        learning_rate_multiplier=0.02,
    )
    asktell.tell(train_data.iloc[0, 0], train_data.iloc[0, 1])
    exp_train_data = train_data.iloc[:N]
    x, y, yhat =  run_ablation_experiment(asktell, exp_train_data, test_data)

    data="iupac-sol"
    model_class="finetune"
    save_csv(out_csv_file, x, y, yhat, data, asktell.get_model_name(), T, k, N, model_class, asktell.tokens_used)

    return y, yhat

In [11]:
N_list=[50,100,250,500,700]
for N in N_list:
  print(f"Running iupac-sol FT with N={N}")
  run_iupac_sol_finetune(train_data, model="text-ada-001", N=N)
  print(" --> done")

Running iupac-sol FT with N=50
running..  Fine-tune succeeded                                                                

ada:ft-white-research-laboratory-2023-02-28-19-10-21
 --> done
Running iupac-sol FT with N=100
running... Fine-tune succeeded                                     -28-19-20-28                                     

ada:ft-white-research-laboratory-2023-02-28-19-20-28
 --> done
Running iupac-sol FT with N=250
running... Fine-tune succeeded                                     -28-19-56-24                                     

ada:ft-white-research-laboratory-2023-02-28-19-56-24
 --> done
Running iupac-sol FT with N=500
running..  Fine-tune succeeded                                                                

ada:ft-white-research-laboratory-2023-02-28-20-41-11
 --> done
Running iupac-sol FT with N=700
running.   Fine-tune succeeded                                     -28-21-30-29                                     

ada:ft-white-research-laboratory-2023-02-28

In [14]:
T_list = [0.05]
k_list = [0]
N_list=[50,100,250,500,700]
models_list = ["text-ada-001"]
for T, k, N, model in itertools.product(T_list, k_list, N_list, models_list):
  print(f"Running iupac-sol finetune ablation with T={T}, k={k}, N={N}, model={model}", end=" ")
  y, yhat = run_iupac_sol_FT_ablation(train_data, test_data, model="text-ada-001", T=T, N=N, k=k)
  print(" --> done")


Running iupac-sol finetune ablation with T=0.05, k=0, N=50, model=text-ada-001  --> done
Running iupac-sol finetune ablation with T=0.05, k=0, N=100, model=text-ada-001  --> done
Running iupac-sol finetune ablation with T=0.05, k=0, N=250, model=text-ada-001  --> done
Running iupac-sol finetune ablation with T=0.05, k=0, N=500, model=text-ada-001  --> done
Running iupac-sol finetune ablation with T=0.05, k=0, N=700, model=text-ada-001  --> done


## Steel alloy

In [34]:
np.random.seed(8)

data_path = "paper/data/yield_strength.csv"
raw_data = pd.read_csv(data_path)

N = raw_data.shape[0]
train = np.random.choice(raw_data.shape[0], int(N * 0.8), replace=False)
test = np.setdiff1d(np.arange(raw_data.shape[0]), train)
np.random.shuffle(test)

train_data = raw_data.iloc[train, :].reset_index(drop=True)
test_data = raw_data.iloc[test, :].reset_index(drop=True)
print(N, len(train_data), len(test_data))

312 249 63


### multi

In [56]:
def run_alloy_multi_ablation(train_data, test_data, model="text-curie-001", T=0.05, N=50, k=10):
    asktell = bolift.AskTellFewShotMulti(
        x_formatter=lambda x: f"alloy composition of {x}",
        y_name="yield strength",
        y_formatter=lambda y: f"{y:.2f}",
        model=model,
        selector_k=k,
        temperature=T
    )
    exp_train_data = train_data[:N]
    x, y, yhat =  run_ablation_experiment(asktell, exp_train_data, test_data)

    data="alloy"
    model_class="multi"
    save_csv(out_csv_file, x, y, yhat, data, model, T, k, N, model_class, asktell.tokens_used)

    return y, yhat


In [57]:

T_list = [0.05]
k_list = [1,2,3,4]
N_list = [249]
for T, k, N, model in itertools.product(T_list, k_list, N_list, models_list):
  print(f"Running alloy multi ablation with T={T}, k={k}, N={N}, model={model}", end=" ")
  y, yhat = run_alloy_multi_ablation(train_data, test_data, model=model, T=T, N=N, k=k)
  print(" --> done")

Running alloy multi ablation with T=0.05, k=1, N=249, model=text-curie-001  --> done
Running alloy multi ablation with T=0.05, k=2, N=249, model=text-curie-001  --> done
Running alloy multi ablation with T=0.05, k=3, N=249, model=text-curie-001  --> done
Running alloy multi ablation with T=0.05, k=4, N=249, model=text-curie-001  --> done


### topk

In [11]:
def run_alloy_topk_ablation(train_data, test_data, model="text-curie-001", T=0.05, N=50, k=10):
    from openai import PromptTemplate
    
    asktell = bolift.AskTellFewShotTopk(
        x_formatter=lambda x: f"alloy composition of {x}",
        y_name="yield strength",
        y_formatter=lambda y: f"{y:.2f}",
        model=model,
        selector_k=k,
        temperature=T
    )
    exp_train_data = train_data[:N]
    x, y, yhat =  run_ablation_experiment(asktell, exp_train_data, test_data)
    data="alloy"
    model_class="topk"
    save_csv(out_csv_file, x, y, yhat, data, model, T, k, N, model_class, asktell.tokens_used)

    return y, yhat


In [ ]:
T_list = [0.05]
k_list = [5]
N_list = [50] #[3,4,5,10,25,50,100,200]
for T, k, N, model in itertools.product(T_list, k_list, N_list, models_list):
  print(f"Running alloy topk ablation with T={T}, k={k}, N={N}, model={model}", end=" ")
  y, yhat = run_alloy_topk_ablation(train_data, test_data, model=model, T=T, N=N, k=k)
  print(" --> done")

### Finetune

In [40]:
def run_alloy_finetune(train_data, model="text-ada-001", N=50):
    asktell = bolift.AskTellFinetuning(
        prefix="",
        prompt_template=PromptTemplate(
            input_variables=["x", "y", "y_name"],
            template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
        ),
        suffix="What is the {y_name} of {x}?@@@\nA:",
        # x_formatter=lambda x: f"alloy composition: {x}",
        y_name="yield strength",
        y_formatter=lambda y: f"{y:.2f}",
        model=model,
        n_epochs=8,
        learning_rate_multiplier=0.02,
    )
    # Tell one example so the moduel build the prompt
    asktell.tell(train_data.iloc[0, 0], train_data.iloc[0, 1])
    exp_train_data = train_data.iloc[:N]

    prompts=[]
    completions=[]
    for i in range(len(exp_train_data)):
        prompts.append(f"What is the yield strength of {exp_train_data.iloc[i, 0]}?@@@\\nA: ")
        completions.append(f"{float(exp_train_data.iloc[i, 1])}###")
    asktell.prepare_data(prompts, completions, f'./paper/out/data_alloy_{N}.dat')
    asktell.fine_tune(prompts, completions, out_path='./paper/out', out_file=f'FT_alloy_{N}')
    print(asktell.get_model_name())

def run_alloy_FT_ablation(train_data, test_data, model="text-ada-001", T=0.05, N=50, k=0):
    with open(f'./paper/out/FT_alloy_{N}.dat', 'r') as f:
        response = json.load(f)
    
    asktell = bolift.AskTellFinetuning(
        prefix="",
        prompt_template=PromptTemplate(
            input_variables=["x", "y", "y_name"],
            template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
        ),
        suffix="What is the {y_name} of {x}?@@@\nA:",
        # x_formatter=lambda x: f"alloy composition: {x}",
        y_name="yield strength",
        y_formatter=lambda y: f"{y:.2f}",
        model=model,
        id=response['id'],
        selector_k=0,
        n_epochs=8,
        learning_rate_multiplier=0.02,
    )
    asktell.tell(train_data.iloc[0, 0], train_data.iloc[0, 1])
    exp_train_data = train_data.iloc[:N]
    x, y, yhat =  run_ablation_experiment(asktell, exp_train_data, test_data)

    data="alloy"
    model_class="finetune"
    save_csv(out_csv_file, x, y, yhat, data, asktell.get_model_name(), T, k, N, model_class, asktell.tokens_used)

    return y, yhat

In [41]:
N_list=[249]
for N in N_list:
  print(f"Running alloy FT with N={N}")
  run_alloy_finetune(train_data, model="text-ada-001", N=N)
  print(" --> done")

Running alloy FT with N=249
text-ada-001
running.   Fine-tune succeeded                                                                

ada:ft-white-research-laboratory-2023-02-27-21-12-15
 --> done


In [43]:
T_list = [0.05]
k_list = [0]
N_list = [249]
models_list = ["text-ada-001"]
for T, k, N, model in itertools.product(T_list, k_list, N_list, models_list):
  print(f"Running alloy finetune ablation with T={T}, k={k}, N={N}, model={model}", end=" ")
  y, yhat = run_alloy_FT_ablation(train_data, test_data, model="text-ada-001", T=T, N=N, k=k)
  print(" --> done")


Running alloy finetune ablation with T=0.05, k=0, N=249, model=text-ada-001 text-ada-001
ada:ft-white-research-laboratory-2023-02-27-21-12-15
 --> done
